# ⚠️ Phase 3 — BNPL Consumer Debt Risk Analysis
### BNPL Under the Microscope

**Objective:** Analyse the consumer-side risk embedded in the BNPL industry.
This is the section that matters most for society — it's what regulators are building policy around.

**Key Questions We Answer:**
1. How do BNPL delinquency rates compare to traditional credit products?
2. Which demographics are most exposed to BNPL debt stress?
3. What is the 'phantom debt' problem and why does it matter systemically?
4. How is late-payment behaviour trending YoY?
5. What's the risk profile of the typical BNPL borrower?

**The Central Tension:**
> BNPL's official default rate (~1.83%) looks great. But 41% of users self-report late payments.
> This is not a contradiction — it's a measurement problem. BNPL's short payment windows,
> auto-debit systems, and non-reporting to credit bureaus create a gap between 'official'
> defaults and true consumer stress. The Richmond Fed calls it 'phantom debt.'
**Sources:**
- CFPB BNPL Market Report, Dec 2025
- CFPB Consumer Use of BNPL, Jan 2025
- NY Fed Consumer Credit Panel, Q1 2025
- Morgan Stanley AlphaWise Survey, Apr 2025
- LendingTree Consumer Survey, 2025
- Richmond Fed Economic Brief EB-25-03

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.ticker as mtick
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

plt.style.use('dark_background')
COLORS = {
    'gold':    '#E8C56D',
    'red':     '#FF5B5B',
    'green':   '#6FCF97',
    'blue':    '#4A9EFF',
    'orange':  '#C45C3A',
    'muted':   '#7A7A8A',
    'bg':      '#0A0A0C',
    'surface': '#111116',
}
plt.rcParams.update({
    'figure.facecolor': COLORS['bg'], 'axes.facecolor': COLORS['surface'],
    'axes.edgecolor': '#2A2A35', 'axes.labelcolor': '#7A7A8A',
    'xtick.color': '#7A7A8A', 'ytick.color': '#7A7A8A',
    'grid.color': '#1E1E28', 'grid.linestyle': '--', 'grid.linewidth': 0.5,
    'font.family': 'monospace', 'font.size': 10, 'figure.dpi': 150,
})

RAW_DIR  = '../data/raw'
PROC_DIR = '../data/processed'
VIS_DIR  = '../visuals'

# Load data
late_pay    = pd.read_csv(f'{RAW_DIR}/bnpl_late_payments_demographics.csv')
delinquency = pd.read_csv(f'{RAW_DIR}/delinquency_comparison.csv')
market_size = pd.read_csv(f'{RAW_DIR}/bnpl_market_size.csv')
print('✅ Data loaded.')

## 3.1 — The Delinquency Paradox

Official BNPL default rates look healthy. Self-reported late payments tell a different story.
Understanding *why* these diverge is key to understanding systemic risk.

In [ ]:
fig, ax = plt.subplots(figsize=(13, 6))

bar_colors = [
    COLORS['green'],   # BNPL official
    COLORS['red'],     # BNPL self-reported
    COLORS['gold'],    # Overall consumer
    COLORS['red'],     # Credit cards
    COLORS['orange'],  # Auto
    COLORS['orange'],  # Student loans
    COLORS['green'],   # Klarna charge-off
]

bars = ax.barh(delinquency['credit_type'], delinquency['delinquency_pct'],
               color=bar_colors, height=0.6, zorder=3)

# Value labels
for bar, val in zip(bars, delinquency['delinquency_pct']):
    ax.text(val + 0.5, bar.get_y() + bar.get_height() / 2,
            f'{val}%', va='center', color='white', fontsize=10, fontweight='bold')

# Source annotations
for i, (_, row) in enumerate(delinquency.iterrows()):
    ax.text(delinquency['delinquency_pct'].max() + 1,
            i, f'  src: {row["source"]}',
            va='center', color=COLORS['muted'], fontsize=7.5, fontstyle='italic')

# Highlight the paradox
ax.annotate('THE PARADOX:\nSame product, 22x gap\nbetween official &\nself-reported rates',
            xy=(41, 1), xytext=(28, 2.2),
            color=COLORS['gold'], fontsize=9,
            arrowprops=dict(arrowstyle='->', color=COLORS['gold'], lw=1),
            bbox=dict(boxstyle='round,pad=0.4', facecolor=COLORS['surface'],
                      edgecolor=COLORS['gold'], alpha=0.9))

ax.set_title('The Delinquency Paradox: BNPL Official vs Self-Reported vs Traditional Credit',
             color='white', fontsize=13, loc='left', pad=14)
ax.set_xlabel('Rate (%)')
ax.grid(True, axis='x', alpha=0.4)
ax.set_xlim(0, 55)

plt.tight_layout()
plt.savefig(f'{VIS_DIR}/05_delinquency_paradox.png', dpi=150, bbox_inches='tight')
plt.show()

print("""
┌─────────────────────────────────────────────────────────────┐
│  WHY THE GAP EXISTS — Key Analyst Finding                   │
│                                                             │
│  BNPL 'official' default rate: ~1.83%  (CFPB, Dec 2025)    │
│  BNPL self-reported late payments: 41% (LendingTree, 2025) │
│                                                             │
│  The gap exists because:                                    │
│  1. BNPL loans are NOT reported to credit bureaus           │
│  2. Auto-debit often masks true payment failures            │
│  3. Short repayment windows = 'missed' vs 'defaulted'       │
│  4. 76% of late payments were <7 days overdue               │
│                                                             │
│  Richmond Fed calls this 'phantom debt' — lenders           │
│  making credit decisions can't see borrowers' BNPL load.   │
└─────────────────────────────────────────────────────────────┘
""")

## 3.2 — Demographic Risk Analysis

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# — Plot 1: Grouped bar — late payments by demographic, 2024 vs 2025
ax = axes[0]
x = np.arange(len(late_pay))
w = 0.36
ax.bar(x - w/2, late_pay['late_payment_pct_2024'], width=w,
       color=COLORS['gold'], alpha=0.55, label='2024', zorder=3)
ax.bar(x + w/2, late_pay['late_payment_pct_2025'], width=w,
       color=COLORS['red'], label='2025', zorder=3)

# YoY delta arrows
for i, row in late_pay.iterrows():
    delta = row['yoy_change_pp']
    ax.annotate(f'+{delta}pp',
                xy=(i + w/2, row['late_payment_pct_2025'] + 1),
                ha='center', color=COLORS['red'], fontsize=9, fontweight='bold')

ax.set_xticks(x)
ax.set_xticklabels(late_pay['demographic'], fontsize=9)
ax.set_title('BNPL Late Payment Rate by Generation (2024 vs 2025)',
             color='white', fontsize=11)
ax.set_ylabel('% reporting late payment')
ax.yaxis.set_major_formatter(mtick.PercentFormatter())
ax.legend(fontsize=9)
ax.grid(True, axis='y', alpha=0.4)

# — Plot 2: Risk profile radar / horizontal stacked
ax = axes[1]
risk_metrics = pd.DataFrame({
    'Metric': [
        'Late payment incidence (2025)',
        'Gen Z late payment rate',
        'Subprime/deep subprime users',
        'Multi-loan stacking (concurrent)',
        'High CC utilisation (>60%)',
    ],
    'Rate': [41, 51, 61, 63, 63],
    'Benchmark': [34, 44, 35, 40, 34],  # prior year or non-BNPL benchmark
})

y_pos = np.arange(len(risk_metrics))
ax.barh(y_pos, risk_metrics['Benchmark'], height=0.55,
        color=COLORS['muted'], alpha=0.4, label='Prior Year / Benchmark', zorder=2)
ax.barh(y_pos, risk_metrics['Rate'], height=0.55,
        color=COLORS['red'], alpha=0.75, label='Current 2025', zorder=3)

# Value labels
for i, (_, row) in enumerate(risk_metrics.iterrows()):
    ax.text(row['Rate'] + 0.5, i, f"{row['Rate']}%",
            va='center', color='white', fontsize=10, fontweight='bold')

ax.set_yticks(y_pos)
ax.set_yticklabels(risk_metrics['Metric'], fontsize=9)
ax.set_title('BNPL Consumer Risk Profile — Current vs Prior Year',
             color='white', fontsize=11)
ax.set_xlabel('% of BNPL Users')
ax.xaxis.set_major_formatter(mtick.PercentFormatter())
ax.set_xlim(0, 75)
ax.legend(fontsize=9)
ax.grid(True, axis='x', alpha=0.4)

plt.suptitle('BNPL Debt Risk — Demographic & Risk Profile Analysis',
             color='white', fontsize=14, y=1.01)
plt.tight_layout()
plt.savefig(f'{VIS_DIR}/06_demographic_risk.png', dpi=150, bbox_inches='tight')
plt.show()
print('💾 Saved: visuals/06_demographic_risk.png')

## 3.3 — BNPL Market Growth vs Risk Divergence

In [ ]:
fig, ax = plt.subplots(figsize=(13, 6))

actuals = market_size[~market_size['is_projected']]
projections = market_size[market_size['is_projected']]

# Area fill
ax.fill_between(actuals['year'], actuals['global_gmv_b'],
                alpha=0.12, color=COLORS['gold'])
ax.fill_between(projections['year'], projections['global_gmv_b'],
                alpha=0.06, color=COLORS['gold'])

# Lines
ax.plot(actuals['year'], actuals['global_gmv_b'],
        color=COLORS['gold'], linewidth=2.5, marker='o', markersize=6, label='Global GMV — Actual')
ax.plot(projections['year'], projections['global_gmv_b'],
        color=COLORS['gold'], linewidth=2.5, marker='o', markersize=6,
        linestyle='--', alpha=0.7, label='Global GMV — Projected')
ax.plot(actuals['year'], actuals['us_market_b'],
        color=COLORS['blue'], linewidth=2, marker='s', markersize=5, label='US Market — Actual')
ax.plot(projections['year'], projections['us_market_b'],
        color=COLORS['blue'], linewidth=2, marker='s', markersize=5,
        linestyle='--', alpha=0.7, label='US Market — Projected')

# Annotation: where risk data starts
ax.axvline(2025, color=COLORS['red'], linewidth=1, linestyle=':', alpha=0.6)
ax.text(2025.05, 550, '← data  projections →',
        color=COLORS['muted'], fontsize=8, va='center')

# Risk warning overlay
ax2 = ax.twinx()
late_pay_trend = pd.DataFrame({
    'year': [2021, 2022, 2023, 2024, 2025],
    'late_pct': [22, 26, 30, 34, 41]
})
ax2.plot(late_pay_trend['year'], late_pay_trend['late_pct'],
         color=COLORS['red'], linewidth=2, marker='^', markersize=7,
         linestyle='-.', label='Users Reporting Late Payments %')
ax2.set_ylabel('Late Payment Rate (%)', color=COLORS['red'])
ax2.tick_params(axis='y', colors=COLORS['red'])
ax2.set_ylim(0, 70)

ax.set_title('BNPL Market Growth vs Rising Consumer Stress — The Divergence',
             color='white', fontsize=13, loc='left', pad=14)
ax.set_ylabel('Market Size (USD Billions)')
ax.set_xlabel('Year')
ax.grid(True, alpha=0.3)

# Combined legend
lines1, labels1 = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax.legend(lines1 + lines2, labels1 + labels2, loc='upper left',
          framealpha=0.2, fontsize=9)

plt.tight_layout()
plt.savefig(f'{VIS_DIR}/07_market_growth_vs_risk.png', dpi=150, bbox_inches='tight')
plt.show()
print('💾 Saved: visuals/07_market_growth_vs_risk.png')

## 3.4 — The Phantom Debt Problem: Systemic Risk Quantification

In [ ]:
# ── Build a model of BNPL's hidden debt exposure ──────────────────────────────

# Known figures from NY Fed Q1 2025 and CFPB reports
total_hh_debt_t = 18.2        # Total US household debt, trillions
credit_card_b   = 1180        # Credit card balances, billions
bnpl_market_b   = 116.7       # US BNPL transaction value 2025, billions
bnpl_users      = 80e6        # ~80M US BNPL users (estimated)
avg_bnpl_balance = 200        # Average first purchase (CFPB), ongoing ~$100/mo

# Estimate active BNPL debt (not GMV, but outstanding balance)
# Typical BNPL: 4 payments over 6 weeks, so ~2/4 outstanding at any time = 50% of GMV
est_outstanding_b = bnpl_market_b * 0.3  # ~30% outstanding at any given point

# Stress scenarios
scenarios = pd.DataFrame({
    'scenario': [
        'Base (official 1.83% charge-off)',
        'Moderate (5% effective default)',
        'Stress (15% in recession)',
        'Severe (20% — 2008-level shock)',
    ],
    'default_rate': [0.0183, 0.05, 0.15, 0.20],
})
scenarios['est_losses_b'] = (scenarios['default_rate'] * est_outstanding_b).round(2)
scenarios['losses_vs_cc_pct'] = (scenarios['est_losses_b'] / (credit_card_b * 0.088) * 100).round(1)

print('━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━')
print('  PHANTOM DEBT STRESS SCENARIOS — US BNPL Market 2025')
print(f'  Estimated US BNPL outstanding balance: ~${est_outstanding_b:.1f}B')
print(f'  Total US household debt: ${total_hh_debt_t}T')
print('━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━')
print(scenarios.to_string(index=False))
print('━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━')
print()
print('KEY INSIGHT:')
print('Even a moderate 5% default rate on BNPL outstanding debt would imply')
print(f'~${scenarios.loc[1, "est_losses_b"]}B in losses — invisible to credit scoring,')
print('undetectable in traditional debt service ratios, and unreported to regulators.')
print()
print('The risk is not the absolute size — it is the opacity.')

scenarios.to_csv(f'{PROC_DIR}/phantom_debt_scenarios.csv', index=False)

In [ ]:
# ── Visualise the stress scenarios ───────────────────────────────────────────
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# — Plot 1: Loss scenarios bar chart
ax = axes[0]
bar_c = [COLORS['green'], COLORS['gold'], COLORS['orange'], COLORS['red']]
bars = ax.barh(scenarios['scenario'], scenarios['est_losses_b'],
               color=bar_c, height=0.55, zorder=3)
for bar, val in zip(bars, scenarios['est_losses_b']):
    ax.text(val + 0.1, bar.get_y() + bar.get_height() / 2,
            f'${val}B', va='center', color='white', fontsize=10, fontweight='bold')
ax.set_title('BNPL Credit Loss Scenarios\n(est. US outstanding ~$35B)',
             color='white', fontsize=11)
ax.set_xlabel('Estimated Losses ($B)')
ax.grid(True, axis='x', alpha=0.4)

# — Plot 2: US debt composition pie + BNPL overlay
ax = axes[1]
debt_data = {
    'Mortgages': 12800,
    'Auto Loans': 1640,
    'Student Loans': 1630,
    'Credit Cards': 1180,
    'Other Consumer': 560,
    'BNPL (est. outstanding)': est_outstanding_b,
}
pie_colors = [
    COLORS['muted'], COLORS['blue'], COLORS['orange'],
    COLORS['gold'], '#4A4A5A', COLORS['red']
]
explode = [0, 0, 0, 0, 0, 0.12]
wedges, texts, autotexts = ax.pie(
    list(debt_data.values()),
    labels=list(debt_data.keys()),
    colors=pie_colors,
    explode=explode,
    autopct=lambda p: f'{p:.1f}%' if p > 2 else '',
    pctdistance=0.75,
    startangle=140,
    wedgeprops={'linewidth': 1, 'edgecolor': COLORS['bg']}
)
for t in texts: t.set_color(COLORS['muted']); t.set_fontsize(8)
for at in autotexts: at.set_color('white'); at.set_fontsize(8)
ax.set_title('US Household Debt Composition\n(BNPL = "invisible" to most credit models)',
             color='white', fontsize=11)

plt.suptitle('The Phantom Debt Problem — Systemic Risk of Unreported BNPL Loans',
             color='white', fontsize=14, y=1.01)
plt.tight_layout()
plt.savefig(f'{VIS_DIR}/08_phantom_debt_risk.png', dpi=150, bbox_inches='tight')
plt.show()
print('\n✅ Phase 3 complete. Proceed to 04_competitor_analysis.ipynb')